In [3]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch
from tqdm import tqdm  # 진행률 표시를 위한 tqdm 라이브러리
from scipy.stats import norm #p-value 를 위함


mirrored_strategy = tf.distribute.MirroredStrategy()
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

2024-05-13 22:31:15.709323: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 22:31:16.079121: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 22:31:17.496449: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/undergrad/miniconda3/envs/tf-2.12.0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook impo

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


2024-05-13 22:31:33.728686: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.8.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'lm_head.weight', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are 

In [86]:
def detection(final_sequence, logit_shape, gamma, delta, token_num, initial_ids) :
    
    input_ids = tokenizer.encode(final_sequence, return_tensors='tf')
    input_ids_list = input_ids.numpy().tolist()[0]
    prev_tk = input_ids[0]
    green_counts = 0
    
    seed = initial_ids[-1]
    np.random.seed(seed)
    indices = np.arange(logit_shape) #0~51999
    green_list_size = int(logit_shape * gamma)  #25600
    green_list = np.random.choice(indices, green_list_size, replace=False)
    print(len(initial_ids[-1]))
    input_ids_list = input_ids_list[len(initial_ids[-1]):]
    print(len(input_ids_list))
    
    num = 0
    for id in input_ids_list :
        
        if id in green_list :
            green_counts = green_counts + 1
        
        seed = id % logit_shape
        #print(f"seed in detection : {seed}")
        np.random.seed(seed)
        
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
       
        num += 1
        

    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")
    green_proportion = green_counts/len(input_ids_list) #u2
    red_proportion = 1-green_proportion 
    print(f"green list 단어 비율 : {green_proportion}")
    print(f"red list 단어 비율 : {red_proportion}")
    #expected_proportion = 0.5 #green list 를 모를 것이다 -> H0 #u1
    #std_error = np.sqrt(expected_proportion * (1 - expected_proportion) / len(input_ids))
    #z_score = 2*(green_counts - len(input_ids_list)/2)/np.sqrt(len(input_ids_list)) #기댓값에서 많이 벗어나면 기각
    z_score = (green_counts - len(input_ids_list)*gamma)/(np.sqrt(len(input_ids_list)*gamma*(1-gamma)))
    #p_value = norm.sf(np.abs(z_score)) #기댓값에 못미치면 green list를 안다!
    
    with open(f'Z_score_g={gamma}_d={delta}.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(str(z_score) + '\n')
            
    print(f"Z-score: {z_score}")
    if z_score > 4 :
        return True
    else :
        return False


In [87]:
filename = 'prompt.txt'
f = open(filename, 'r')
lines = f.readlines()
print(lines)
print('-' * 10)

['아침 일과 시작하기\n', '책 읽기의 즐거움\n', '가족과 함께 하는 저녁 식사\n', '친구와의 영화 관람\n', '주말 동안의 여행 계획\n', '정원 가꾸기의 효과\n', '요리를 통한 스트레스 해소\n', '자전거 타기의 장점\n', '집안 정리 정돈 팁\n', '개인 운동 루틴 유지하기\n', '취미 활동의 중요성\n', '새로운 언어 배우기\n', '음악 연주와 그 영향\n', '사진 촬영을 통한 추억 만들기\n', '지역 사회 봉사의 보람\n', '패션과 개인 스타일 표현\n', '휴일을 활용한 가족 모임\n', '건강한 식생활을 위한 조리법\n', '애완동물과 보내는 시간\n', '독서 클럽에서의 토론\n', '도서관 방문의 장점\n', '집에서의 필름 감상\n', '커피 한 잔과의 여유\n', '일기 쓰기의 치유 효과\n', '홈 베이킹의 즐거움\n', '직접 만든 공예품\n', '새로운 요리법 시도하기\n', '지역 공연 관람\n', '자연 속에서의 하이킹\n', '가구 리폼 프로젝트\n', '아이와 함께하는 공원 나들이\n', '새로운 스포츠 도전\n', '개인 기록의 개선\n', '집에서의 요가 세션\n', '반려동물과의 산책\n', '가족 앨범 만들기\n', '매일의 명상 시간\n', '가벼운 아침 조깅\n', '주말 브런치 준비\n', '일상의 소소한 행복 찾기\n', '스마트폰 없는 하루\n', '책상 정리의 중요성\n', '옷장 정리하기\n', '새벽 시간의 고요함\n', '오후의 티타임\n', '자기 전 명상\n', '집안 식물 가꾸기\n', '근처 박물관 탐방\n', '집에서의 칵테일 파티\n', '커뮤니티 클래스 참여\n', '셀프 케어 루틴\n', '장거리 걷기의 이점\n', '텃밭 가꾸기\n', '개인적인 스타일 개발\n', '책 추천 공유\n', '로컬 마켓 쇼핑\n', '장기 게임의 즐거움\n', '가족과의 보드게임 밤\n', '디지털 디톡스\n', '가까운 도시 탐험\n', '수제 비누 만들기\n', '로컬 아

In [26]:
# Function to apply dynamic repetition penalty to logits
def apply_repetition_penalty(logits, token_counts, penalty_coefficient):
    penalty_factors = np.ones(logits.shape[-1])

    for token_id, count in enumerate(token_counts):
        if count > 0:
            penalty_factors[token_id] = penalty_coefficient ** count

    # Apply the calculated penalties to logits
    return logits / penalty_factors

In [88]:

def detection_func(param_d, param_g, token_num, penalty_coefficient):

  with open(f'Z_score_g={param_g}_d={param_d}.txt', 'w') as f:  #file내용 clear
      f.write('')

  with open(f'output_g={param_g}_d={param_d}.txt', 'w') as f:  #file내용 clear
      f.write('')
      
  for line in lines:
      line = line.strip()
      print(f"prompt : {line}")

      input_text = line
      input_ids = tokenizer.encode(input_text, return_tensors='tf')
      initial_ids = input_ids

      with open(f'output_g={param_g}_d={param_d}.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(f'{str(len(input_ids[-1]))}' + '\n') #한 문장 토큰 앞서서, input 토큰 개수 
        f.write(f'{str(token_num)}' + '\n') #한 문장 토큰 앞서서, 생성 토큰 개수 
        for i in range(len(input_ids[-1])) :
          f.write(str(input_ids.numpy()[0][i]) + '\n')

      last_index = input_ids.shape[-1]
      nlls = []
      token_counts = np.zeros(tokenizer.vocab_size)
              
      with mirrored_strategy.scope():
          for _ in tqdm(range(token_num)):  # 최대 100번 반복
              #if input_ids.shape[-1] >= token_num:
              #    break  # input_ids의 길이가 100 이상이면 반복 중단
              
              #print("================================")
              #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
              #set 나눠서 green set 로짓을 촉진하기
              gamma = param_g
              delta = param_d
              Logits = model(input_ids).logits
              #print(f"original Logits : {Logits}")
              
              #seed 생성
              seed = input_ids[-1] % Logits.shape[-1]
              #print(f"seed : {seed[-1]}")
              np.random.seed(seed[-1])

              #random green list 설정
              indices = np.arange(Logits.shape[-1]) #0~51999
              green_list_size = int(Logits.shape[-1] * gamma)  #25600
              green_list = np.random.choice(indices, green_list_size, replace=False)
              #print(f"green list : {green_list} {green_list[200]} {green_list[1000]}") #25600개의 green list

              #--여기부터 probability 구하기
              # green logit과 red logit의 값을 담을 리스트 생성
              green_logits = tf.zeros(green_list_size)
              red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

              #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
              Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
              Logits_num = Logits.numpy()
              Logits_num[0, -1, :] = apply_repetition_penalty(Logits_num[0, -1, :], token_counts, penalty_coefficient)
              
              for i in green_list :
                  Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
              Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
              #print(f"updated Logits : {Logits}")
              
              probs = tf.nn.softmax(Logits[0,last_index-1,:])

              add_token_id = tf.argmax(probs)
              #print(f"max : {max(probs)}")
              
              token_counts[add_token_id] += 1
              
              selected_probability = probs[add_token_id]
              nlls.append(np.log(selected_probability))
              
              #print(softmax_tensor[0, last_index-1, add_token_id])

              # 텐서 연결
              add_token_id = tf.reshape(add_token_id, [1,1])
              add_token_id = tf.cast(add_token_id, input_ids.dtype)
              input_ids = tf.concat([input_ids, add_token_id], axis=-1)
              next_token = tokenizer.decode(input_ids.numpy()[0,:])
              with open(f'output_g={param_g}_d={param_d}.txt', 'a') as f:
                f.write(str(add_token_id[0][0].numpy()) + '\n')
              #print(f"next token : {next_token}")
              
              last_index = input_ids.shape[-1]
              #term = 3
          final_token = tokenizer.decode(input_ids.numpy()[0,:])
          print(f"final sequence : {final_token}")
          print(f"detection function : {detection(final_token, Logits.shape[-1], gamma, delta, token_num, initial_ids)}")

In [210]:
detection_func(0.5,0.1, 100, 1.2)

prompt : 아침 일과 시작하기


100%|██████████| 100/100 [00:14<00:00,  6.96it/s]


final sequence : 아침 일과 시작하기 전에 미리 준비해두면 좋다.
아침식사는 아침과 저녁으로 나눠서 한다.
저녁은 밥 한 공기를 먹고, 점심은 반찬이랑 같이 먹는다.
점심시간에는 식사 후 간단한 간식을 챙겨먹는다.
오후 2시 30분부터 4시까지는 잠자리에 들기 전 식사를 하고 오후 5시부터 6시까지 자야한다.
잠을 잘 때 가장 중요한 것은 규칙적인 식습관이다.
규칙적인 생활로 인해 체중이 늘거나 과체중일 경우엔 반드시 전문의와 상의
4
100
green list 단어 개수 : 11
red list 단어 개수 : 89
green list 단어 비율 : 0.11
red list 단어 비율 : 0.89
Z-score: 0.3333333333333333
detection function : False
prompt : 책 읽기의 즐거움


100%|██████████| 100/100 [00:14<00:00,  6.96it/s]


final sequence : 책 읽기의 즐거움도 함께 느낄 수 있다.
이번 행사는 오는 27일까지 진행되며, 참가비는 무료다.
참가 신청은 이메일(kimsang<unk>gmail.com)로 하면 된다.
한편 이번 행사에서는 '2017 대한민국 독서대전' 수상작 전시회도 열린다.
올해로 6회째를 맞는 대한민국 독서대전은 문화체육관광부가 주최하고 한국출판문화산업진흥원이 주관하는 행사로 올해는 총 10개 부문으로 나눠 진행된다.
특히, 올해의 경우 지난해보다 한 단계 업그레이드된
5
100
green list 단어 개수 : 23
red list 단어 개수 : 77
green list 단어 비율 : 0.23
red list 단어 비율 : 0.77
Z-score: 4.333333333333333
detection function : True
prompt : 가족과 함께 하는 저녁 식사


...

100%|██████████| 100/100 [00:14<00:00,  6.80it/s]


final sequence : 주변 환경 정화 활동 등을 통해 환경보호 활동을 지속적으로 펼쳐나갈 계획이다.
한편 지난해에는 총 3억원의 사업비를 투입, 관내 초등학교 주변과 학교 주변에 대한 정화활동을 펼쳤다.</d> 서울중앙지검 특수2부(부장검사 송경호)는 이날 오전 10시30분부터 오후 1시까지 김 전 차관을 피의자 신분으로 불러 조사하고 있다.
김 전 차관 측은 "검찰에서 진술할 내용이 많아 오늘은 출석하지 않을 것"이라며 말을 아꼈다.
앞서 검찰 특별수사
4
100
green list 단어 개수 : 13
red list 단어 개수 : 87
green list 단어 비율 : 0.13
red list 단어 비율 : 0.87
Z-score: 1.0
detection function : False
prompt : 하루의 목표 세우기


100%|██████████| 100/100 [00:14<00:00,  6.84it/s]


final sequence : 하루의 목표 세우기다.
이번 캠페인은 지난해 12월 31일부터 올 1월 1일까지 진행됐다.
캠페인 기간 동안 총 1만5000여명이 참여해 약 4억원의 매출을 올렸다.
특히, 이번 캠프에서는 ‘2017 대한민국 동행세일’ 행사에서 판매된 상품 중 가장 높은 수익률을 기록한 상품을 선정했다.
‘대한민국 동행세일’은 코로나19로 인해 어려움을 겪고 있는 소상공인과 자영업자를 지원하기 위해 기획한 행사로 전국 주요 백화점과 대형마트, 온라인 쇼핑몰 등에서
4
100
green list 단어 개수 : 12
red list 단어 개수 : 88
green list 단어 비율 : 0.12
red list 단어 비율 : 0.88
Z-score: 0.6666666666666666
detection function : False
prompt : 온라인 학습 코스


 52%|█████▏    | 52/100 [00:07<00:07,  6.83it/s]

### Perplexity 계산

In [ ]:
def calculate_perplexity(param_d, param_g, prompt_num):

    with open(f'perplexity_g={param_g}_d={param_d}.txt', 'w') as f:  #file내용 clear
        f.write('')

    with open(f'output_g={param_g}_d={param_d}.txt', 'r') as file:
        token_ids = [int(line.strip()) for line in file.readlines()]
    
    for _ in range(prompt_num) :
      prompt_size = int(token_ids[0])
      token_num = int(token_ids[1])

      # Prepare to collect nll
      log_probs = []

      # Calculate the probability for each token in the list
      for k in tqdm(range(prompt_size+3, prompt_size + token_num+3)):
          # Prepare the input for the model: all previous tokens up to the k-th token
          input_ids = tf.convert_to_tensor([token_ids[0:k]], dtype=tf.int32)
          
          # Get the logits from the model
          outputs = model(input_ids)
          logits = outputs.logits
          
          # Select the logits for the last position (k-th token prediction)
          last_logits = logits[0, -1, :]
          
          # Apply softmax to convert logits to probabilities
          token_probabilities = tf.nn.softmax(last_logits, axis=-1)
          
          # Get the probability of the k-th token
          k_token_id = token_ids[k]
          k_probability = token_probabilities[k_token_id].numpy()

          # Calculate the log probability
          log_prob = np.log(k_probability)
          log_probs.append(log_prob)
      
      # Calculate the mean of the negative log probabilities
      mean_nll = -np.mean(log_probs)
        
      # Calculate perplexity
      perplexity = np.exp(mean_nll)
      print(f"perplexity : {perplexity}")

      with open(f'perplexity_g={param_g}_d={param_d}.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(str(perplexity) + '\n')

      #다음 프롬프트
      if prompt_size + token_num + 2 <= len(token_ids) :
          token_ids = token_ids[prompt_size + token_num + 2:]

In [ ]:
calculate_perplexity(0.5,0.1,100)

NameError: name 'calculate_perplexity' is not defined

#### Token에 따른 detection_func

In [186]:
def detection_func_for_token(final_sequence, logit_shape, gamma, delta, token_num, initial_ids) :
    
    input_ids_list = final_sequence[len(initial_ids[-1]):]
    green_counts = 0
    
    seed = initial_ids[-1]
    np.random.seed(seed)
    indices = np.arange(logit_shape) #0~51999
    green_list_size = int(logit_shape * gamma)  #25600
    green_list = np.random.choice(indices, green_list_size, replace=False)
    
    num = 0
    for id in input_ids_list :
        
        if id in green_list :
            green_counts = green_counts + 1
        #else :
        #    print(tokenizer.decode(id))
        
        seed = id % logit_shape
        #print(f"seed in detection : {seed}")
        np.random.seed(seed)
        
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
       
        num += 1
        

    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")
    green_proportion = green_counts/len(input_ids_list) #u2
    red_proportion = 1-green_proportion 
    #print(f"green list 단어 비율 : {green_proportion}")
    #print(f"red list 단어 비율 : {red_proportion}")
    #expected_proportion = 0.5 #green list 를 모를 것이다 -> H0 #u1
    #std_error = np.sqrt(expected_proportion * (1 - expected_proportion) / len(input_ids))
    #z_score = 2*(green_counts - len(input_ids_list)/2)/np.sqrt(len(input_ids_list)) #기댓값에서 많이 벗어나면 기각
    z_score = (green_counts - len(input_ids_list)*gamma)/(np.sqrt(len(input_ids_list)*gamma*(1-gamma)))
    #p_value = norm.sf(np.abs(z_score)) #기댓값에 못미치면 green list를 안다!
    
    with open(f'Z_score_g={gamma}_d={delta}_token.txt', 'a') as f:  # with 문을 사용하여 파일을 열면 자동으로 닫힙니다.
        f.write(str(z_score) + '\n')
            
    print(f"Z-score: {z_score}")
    if z_score > 4 :
        return True
    else :
        return False


In [199]:
def token_detection(delta, gamma, prompt_num) :
      with open(f'output_g={gamma}_d={delta}.txt', 'r') as file:
            token_ids = [int(line.strip()) for line in file.readlines()]
      #print(tokenizer.decode(token_ids[0:]))
            
      with open(f'Z_score_g={gamma}_d={delta}_token.txt', 'w') as f:
            f.write('')
            
      for _ in range(0,1) :
            prompt_size = int(token_ids[0])
            token_num = int(token_ids[1])
            token_ids = token_ids[prompt_size + token_num + 2:]
            initial_ids = token_ids[2:2+prompt_size]
            final_token = token_ids[2:prompt_size + token_num+2]
            print(initial_ids)
            print(final_token)
            
            for i in tqdm(range(1,prompt_size+token_num+1)) :
                  if i>prompt_size :
                        print(i)
                        detection_func_for_token(final_token[0:i], 51200, gamma, delta, i, [initial_ids])

            #다음 프롬프트
            if prompt_size + token_num + 2 <= len(token_ids) :
                  token_ids = token_ids[prompt_size + token_num + 2:]

In [209]:
token_detection(5.0,0.9,100)

[9450, 11088, 9480, 20244]
[9450, 11088, 9480, 20244, 8098, 7235, 9304, 20166, 9025, 9029, 8146, 7623, 10647, 7162, 9194, 8347, 26347, 20244, 8098, 12141, 16727, 9036, 9882, 13230, 6903, 9494, 11328, 9021, 9389, 14465, 14453, 9052, 9450, 11088, 9480, 28881, 9103, 7049, 8705, 9025, 9080, 11153, 8148, 9194, 8347, 7607, 17196, 19534, 8563, 7428, 25843, 9233, 8429, 8643, 9225, 10488, 7249, 9029, 7335, 8704, 39735, 10741, 10251, 31412, 11204, 387, 39848, 9030, 35533, 20738, 8719, 18649, 24733, 26421, 22301, 10647, 9180, 13216, 36766, 9277, 9553, 6918, 10946, 26655, 37484, 9207, 12846, 9023, 9939, 9855, 50490, 41730, 18567, 9033, 15631, 26026, 8187, 9140, 19534, 8563, 7428, 13065, 10001, 9156]


 17%|█▋        | 18/104 [00:00<00:00, 174.15it/s]

5
green list 단어 개수 : 1
red list 단어 개수 : 0
Z-score: 0.33333333333333326
6
green list 단어 개수 : 2
red list 단어 개수 : 0
Z-score: 0.4714045207910316
7
green list 단어 개수 : 3
red list 단어 개수 : 0
Z-score: 0.5773502691896255
8
green list 단어 개수 : 4
red list 단어 개수 : 0
Z-score: 0.6666666666666665
9
green list 단어 개수 : 5
red list 단어 개수 : 0
Z-score: 0.74535599249993
10
green list 단어 개수 : 6
red list 단어 개수 : 0
Z-score: 0.8164965809277257
11
green list 단어 개수 : 7
red list 단어 개수 : 0
Z-score: 0.8819171036881972
12
green list 단어 개수 : 8
red list 단어 개수 : 0
Z-score: 0.9428090415820632
13
green list 단어 개수 : 9
red list 단어 개수 : 0
Z-score: 1.0000000000000004
14
green list 단어 개수 : 10
red list 단어 개수 : 0
Z-score: 1.0540925533894598
15
green list 단어 개수 : 11
red list 단어 개수 : 0
Z-score: 1.105541596785133
16
green list 단어 개수 : 12
red list 단어 개수 : 0
Z-score: 1.154700538379251
17
green list 단어 개수 : 13
red list 단어 개수 : 0
Z-score: 1.2018504251546622
18
green list 단어 개수 : 14
red list 단어 개수 : 0
Z-score: 1.2472191289246477
19
green 

 35%|███▍      | 36/104 [00:00<00:00, 78.27it/s] 

green list 단어 개수 : 32
red list 단어 개수 : 0
Z-score: 1.8856180831641265
37
green list 단어 개수 : 33
red list 단어 개수 : 0
Z-score: 1.9148542155126769
38
green list 단어 개수 : 34
red list 단어 개수 : 0
Z-score: 1.9436506316150997
39
green list 단어 개수 : 35
red list 단어 개수 : 0
Z-score: 1.972026594366539
40
green list 단어 개수 : 36
red list 단어 개수 : 0
Z-score: 2.000000000000001
41
green list 단어 개수 : 37
red list 단어 개수 : 0
Z-score: 2.027587510099404
42
green list 단어 개수 : 38
red list 단어 개수 : 0
Z-score: 2.0548046676563243
43
green list 단어 개수 : 39
red list 단어 개수 : 0
Z-score: 2.081665999466132
44


...

99
green list 단어 개수 : 95
red list 단어 개수 : 0
Z-score: 3.2489314482696545
100
green list 단어 개수 : 96
red list 단어 개수 : 0
Z-score: 3.265986323710903
101


 98%|█████████▊| 102/104 [00:03<00:00, 14.96it/s]

green list 단어 개수 : 97
red list 단어 개수 : 0
Z-score: 3.282952600598703
102
green list 단어 개수 : 98
red list 단어 개수 : 0
Z-score: 3.2998316455372207
103
green list 단어 개수 : 99
red list 단어 개수 : 0
Z-score: 3.316624790355397
104


100%|██████████| 104/104 [00:03<00:00, 27.61it/s]

green list 단어 개수 : 100
red list 단어 개수 : 0
Z-score: 3.333333333333334


### Output Token 색깔 확인

In [81]:
def detection_output_token_color(final_sequence, logit_shape, gamma, initial_ids) :
    
    input_ids_list = final_sequence
    green_counts = 0
    
    seed = initial_ids[-1]
    np.random.seed(seed)
    indices = np.arange(logit_shape) #0~51999
    green_list_size = int(logit_shape * gamma)  #25600
    green_list = np.random.choice(indices, green_list_size, replace=False)
    
    num = 0
    green_token_represent = []
    
    for id in input_ids_list :
        
        if id in green_list :
            green_counts = green_counts + 1
            green_token_represent.append(tokenizer.decode(id))
        
        seed = id % logit_shape
        #print(f"seed in detection : {seed}")
        np.random.seed(seed)
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
       
        num += 1
        
    print(f"green token sequence : {green_token_represent}")
    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")


In [82]:
with open(f'test_output.txt', 'r') as file:
    token_ids = [int(line.strip()) for line in file.readlines()]
final_sequence = token_ids
print(tokenizer.decode(final_sequence))
print(final_sequence)
initial_ids = final_sequence[0:4]
detection_output_token_color(final_sequence, 51200, 0.5, initial_ids) 

책 읽기의 즐거움도 함께 느낄 것이다.
이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다.
책읽기의 즐거움뿐만 아니라, 책을 통해 삶의 지혜와 지혜를 배울 수 있는 자리다.
김씨는 "'책은 읽는 사람'과 '책을 읽어야만 사는 사람'의 차이를 이해하고 책 읽는 사람의 마음을 이해하는 것이 중요하다”며 “이런 점에서 이번에 마련한 특별전은 독서문화의 새로운 지평(知평을 넓힘으로써, 책과 사람을 잇는 길을 제시해 준다”고 말했다.
한편 서울시는 오는
[9450, 11088, 9480, 20244, 8098, 7235, 9304, 20166, 9257, 8146, 7623, 10647, 7162, 9194, 28175, 14757, 14140, 9367, 12951, 14283, 22651, 10892, 8148, 9324, 8746, 7848, 15389, 18992, 8347, 8150, 9480, 20244, 8098, 11314, 12105, 13216, 9430, 13368, 48730, 30528, 35315, 9025, 9080, 10300, 9017, 6963, 16372, 10063, 382, 16095, 26347, 9179, 17196, 9194, 12115, 11088, 11992, 7489, 11594, 9179, 12479, 14632, 20771, 9450, 26347, 10829, 13489, 17450, 9199, 9861, 8702, 15910, 7530, 22137, 7399, 10943, 9018, 18554, 30175, 10228, 11865, 19534, 15820, 9689, 9027, 13825, 4267, 15558, 9849, 45858, 387, 39735, 11389, 14140, 13165, 10657, 8711, 9701, 26973, 13525, 8704, 8652, 9563, 11768, 14472]
green token sequence : ['책', '즐거', '움', '도',

#### Output으로 Z-score만 계산하기

In [109]:

def calculate_only_zscore(param_d, param_g, token_num, penalty_coefficient, prompt_num):

    with open(f'Z_score_g={param_g}_d={param_d}.txt', 'w') as f:  #file내용 clear
        f.write('')
      
    with open(f'output_g={param_g}_d={param_d}.txt', 'r') as file:
        token_ids = [int(line.strip()) for line in file.readlines()]
      
    for _ in range(prompt_num) :
        prompt_size = int(token_ids[0])
        token_num = int(token_ids[1])
        initial_ids = token_ids[2:2+prompt_size]
        final_token = token_ids[2:prompt_size + token_num+2]
        final_token = tokenizer.decode(final_token)
        print(f"final sequence : {final_token}")
        print(f"detection function : {detection(final_token, 51200, param_g, param_d, token_num, [initial_ids])}")
        
        if prompt_size + token_num + 2 <= len(token_ids) :
            token_ids = token_ids[prompt_size + token_num + 2:]

In [121]:
calculate_only_zscore(0.5, 0.1, 100, 1.2, 100)

FileNotFoundError: [Errno 2] No such file or directory: 'output_g=0.1_d=0.5.txt'